In [7]:
import com.github.ax_as.datapro.data.*
import java.nio.file.Path

fun toLocalPath(file: String) = Path.of("/data/files/output/$file")
val htmlSerproBB = toLocalPath("serpro_vs_bb.html")

val configuration = DisplayConfiguration(
    rowsLimit = null,
    useDarkColorScheme = true,
)


var bb = DataFrame.concursoBB(DataFiles.bb)
var index = 1


var listaAc =
    DataFrame.concursoSerpro(DataFiles.serpro_ac).sortByClassificacaoFinal()
        .add("lista") { "AC" }.move("lista").to(1).join(bb, JoinType.Left) {
            it["nome"] match right["nome"]
        }.add("pos_ex") {
            it["situacao_final"]?.toString().let {
                val v = if (it == "CLASSIFICADO") "" else "${index++}"
                v
            }
        }.move("pos_ex").to(1)
        .fillNA {
            colsOf<Number?>()
        }.withZero().fillNA { all() }.with { "" }



index = 1
var listaPPP =
    DataFrame.concursoSerpro(DataFiles.serpro_ppp).sortByClassificacaoFinal()
        .add("lista") { "PPP" }.move("lista").to(1).join(bb, JoinType.Left) {
            it["nome"] match right["nome"]
        }.add("pos_ex") {
            it["situacao_final"]?.toString().let {
                val v = if (it == "CLASSIFICADO") "" else "${index++}"
                v
            }
        }.move("pos_ex").to(1)
        .fillNA {
            colsOf<Number?>()
        }.withZero().fillNA { all() }.with { "" }

index = 1
var listaPcd =
    DataFrame.concursoSerpro(DataFiles.serpro_pcd).sortByClassificacaoFinal()
        .add("lista") { "PCD" }.move("lista").to(1).join(bb, JoinType.Left) {
            it["nome"] match right["nome"]
        }.add("pos_ex") {
            it["situacao_final"]?.toString().let {
                val v = if (it == "CLASSIFICADO") "" else "${index++}"
                v
            }
        }.move("pos_ex").to(1)
        .fillNA {
            colsOf<Number?>()
        }.withZero().fillNA { all() }.with { "" }


index = 1
var pratica_ac =
    DataFrame.praticaSerpro(DataFiles.serpro_final_pratica, "AC")
        .add("pos_pratica") { index++ }

index = 1
var pratica_ppp =
    DataFrame.praticaSerpro(DataFiles.serpro_final_pratica_ppp, "PPP")
        .add("pos_pratica") { index++ }

index = 1
var pratica_pcd =
    DataFrame.praticaSerpro(DataFiles.serpro_final_pratica_pcd, "PCD")
        .add("pos_pratica") { index++ }

index = 1
var provisoria_ac =
    DataFrame.praticaSerpro(
        DataFiles.serpro_prov_pratica,
        "AC",
        header = headerPraticaProv
    )
        .add("pos_pratica") { index++ }

index = 1
var provisoria_pcd =
    DataFrame.praticaSerpro(
        DataFiles.serpro_prov_pratica_pcd,
        "PCD",
        header = headerPraticaProv
    )
        .add("pos_pratica") { index++ }

index = 1
var provisoria_ppp =
    DataFrame.praticaSerpro(
        DataFiles.serpro_prov_pratica_ppp,
        "PPP",
        header = headerPraticaProv
    )
        .add("pos_pratica") { index++ }

index = 1

var acByPontosGanhos = pratica_ac.join(provisoria_ac, JoinType.Left) {
    it["inscricao"] match right["inscricao"]
}.add("pontos_ganhos_recurso") {
    "nota_pratica"<Float>() - "nota_pratica_prov"<Float>()
}.select(columns = headerPraticaPontosGanhos)
    .sortByDesc("pontos_ganhos_recurso")

var pppByPontosGanhos = pratica_ppp.join(provisoria_ppp, JoinType.Left) {
    it["inscricao"] match right["inscricao"]
}.add("pontos_ganhos_recurso") {
    "nota_pratica"<Float>() - "nota_pratica_prov"<Float>()
}.select(columns = headerPraticaPontosGanhos)
    .sortByDesc("pontos_ganhos_recurso")

var pcdByPontosGanhos = pratica_pcd.join(provisoria_pcd, JoinType.Left) {
    it["inscricao"] match right["inscricao"]
}.add("pontos_ganhos_recurso") {
    "nota_pratica"<Float>() - "nota_pratica_prov"<Float>()
}.select(columns = headerPraticaPontosGanhos)
    .sortByDesc("pontos_ganhos_recurso")

var allSerpro = listaAc.concat(listaPPP, listaPcd)
var allPratica = pratica_ac.concat(pratica_ppp, pratica_pcd)

allSerpro
    .toDefaultHtml()
    .writeHTML(htmlSerproBB)

index = 1
acByPontosGanhos.concat(pppByPontosGanhos, pcdByPontosGanhos)
    .sortByDesc("pontos_ganhos_recurso")
    .distinctBy("nome")
    .add("pos") { index++ }.toDefaultHtml()
    .writeHTML(toLocalPath("by_pontos_ganhos_completa.html"))




